# Custom RAG Pipeline w/ History Implementation

This program will be adding history to our RAG pipeline, modifying history from built in RAG history retrievers to creating custom chains with the "|" operator. Creating a custom retriever allows us better control over context injection, injecting in smaller chunks with metadata chunks.

In [1]:
import os
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

In [2]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [3]:
def load_docs():
    
    document_loader = []

    for root, dirs, files in os.walk("."):
        # Skip chroma_db folder
        if "faiss" in root or "git" in root:
            continue
        for file in files:
            if file.endswith(".pdf"):
                document_loader.append(file)

    return document_loader

In [4]:
document_loader = load_docs()
document_loader

['ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 'Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 'ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf',
 'Three Phase Power System Fundamentals.pdf']

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model.save("./local_models/all-MiniLM-L6-v2")

In [12]:
embedding_model ="./local_models/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file in document_loader:
        loader = PyPDFLoader(file)
        doc = loader.load()
        doc_store += doc

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits

In [13]:
embeddings, splits = embed_splitting(document_loader, embedding_model)

In [14]:
example_split = splits[106]
example_split

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily underst

In [16]:
metadata = example_split.metadata
for key in metadata:
    print(f"{key}: {metadata[key]}")

producer: Microsoft® Word 2013
creator: Microsoft® Word 2013
creationdate: 2019-07-27T15:04:48+08:00
author: Ali Kharrazi
moddate: 2019-07-27T15:04:48+08:00
source: ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf
total_pages: 76
page: 51
page_label: 52


In [17]:
embeddings

HuggingFaceEmbeddings(model_name='./local_models/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
len(splits)

402

In [19]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [20]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [21]:
# create the retriever object once
semantic_retriever = vector_store.as_retriever(search_kwargs={'k': 4})

# define your function to query it
def semantic_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

# call the function with retriever and query string
results = semantic_search(semantic_retriever, "Explain transformers")

In [22]:
results[0].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2019-07-27T15:04:48+08:00',
 'author': 'Ali Kharrazi',
 'moddate': '2019-07-27T15:04:48+08:00',
 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'total_pages': 76,
 'page': 51,
 'page_label': '52'}

In [23]:
for i in range(len(results)):
    source_data = results[i].metadata["source"]
    page = results[i].metadata["page"]
    page_content = results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 51. 

 The page content is Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shel

In [24]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 4

def keyword_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

In [25]:
keyword_results = keyword_search(bm25_retriever, "Explain transformers")

In [26]:
keyword_results

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}, page_content='65 Electrical Machines and Systems                                                                                                            \n6.8 Current Transformers \nInstrument transformers are special transformers for extending the range of measur ing instruments. \nThere are two basic types: voltage transformers for measuring high voltages, and current transformers \nfor measuring high currents. Using transformers for voltage measurement is similar in principle to the \nordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current \ntransformers, on the other hand, need special consideration. These are

In [27]:
for i in range(len(keyword_results)):
    source_data = keyword_results[i].metadata["source"]
    page = keyword_results[i].metadata["page"]
    page_content = keyword_results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 64. 

 The page content is 65 Electrical Machines and Systems                                                                                                            
6.8 Current Transformers 
Instrument transformers are special transformers for extending the range of measur ing instruments. 
There are two basic types: voltage transformers for measuring high voltages, and current transformers 
for measuring high currents. Using transformers for voltage measurement is similar in principle to the 
ordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current 
transformers, on the other hand, need special consideration. These are usually toroidal transformers with 
high-quality core material. 
Figure 6-25 shows a load connected to a source. The primary of a current transformer is in series with 
the load, and the secondary 

In [28]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers= [semantic_retriever, bm25_retriever], weights = [0.6, 0.4])

In [29]:
combined_results = ensemble_retriever.invoke("Explain transformers")

In [30]:
combined_results

[Document(id='edb5304c-c167-48dc-b7e2-e01bb20967fe', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [31]:
for i in combined_results:

    print(i.metadata)

{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 50, 'page_label': '51'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 52, 'page_label': '53'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'

In [32]:
i = 1
result_list = [combined_results[0]]
seen_pages = {combined_results[0].metadata["page_label"]}

while i < len(combined_results):
    metadata = combined_results[i].metadata
    page_label = metadata["page_label"]

    if page_label in seen_pages:
        i += 1  # You MUST increment i here
        continue

    result_list.append(combined_results[i])
    seen_pages.add(page_label)
    i += 1

In [33]:
result_list

[Document(id='edb5304c-c167-48dc-b7e2-e01bb20967fe', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [34]:
input_template = """You are an expert assistant answering based only on the provided context.

The retrieved documents have been joined together and are separated by "Chunk_n", where n is the chunk number. Here is the context:

{context}

Use all relevant information above to answer the question below. If the answer isn't found in the chunks, say:
"I cannot answer this question because the necessary information was not found in the provided documents."

❗Do not cite or mention any source files or page numbers in the body of your answer.

At the end of your answer, add a single line in this format:

Information was pulled from: <source_file_1>: pages <comma-separated page numbers>; <source_file_2>: pages <...>; ...

Use only one entry per document, listing all unique page numbers where information was pulled from.
Do not mention metadata_n, chunk_n, or include references in the main answer.

Metadata:
{metadata}

Question: {question}
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableParallel

def content_parser(results):
    context_string = ""
    for i in range(len(results)):

        context_string += f"\nChunk_{i+1}:\n\n{results[i].page_content}\n\n\n"

    return context_string 

chunks = content_parser(result_list)

In [ ]:
chunk_runnable = RunnableLambda(content_parser)

In [36]:
def metadata_parser(results):
    metadata_files = {}

    for doc in results:
        source = doc.metadata["source"]
        page = doc.metadata["page_label"]

        if source in metadata_files:
            if page not in metadata_files[source]:
                metadata_files[source].append(page)
        else:
            metadata_files[source] = [page]

    metadata_string = "This file uses the following sources:"

    for key in metadata_files:
        pages = ", ".join(str(i) for i in metadata_files[key])
        metadata_string += f"\n\n{key}, pages {pages}"
        

    return metadata_string

In [50]:
metadata_runnable = RunnableLambda(metadata_parser)
print(metadata_runnable.invoke(result_list))

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 51, 53, 56, 65, 69


In [37]:
metadata = metadata_parser(result_list)

In [38]:
print(metadata)

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 51, 53, 56, 65, 69


In [40]:
prompt_template = ChatPromptTemplate(
    [
        ("system", input_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference previous context,
reformulate it as a fully standalone question that can be answered without the chat history.
If the question is already standalone, return it as-is."""

contextual_prompt = ChatPromptTemplate(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [ ]:
chain = prompt_template | llm
answer1 = chain.invoke(
    {'context': chunks,
     'metadata': metadata,
     'chat_history': [],
     'input': "Explain transformers",
     'question': "Explain transformers"}
)

In [101]:
history = []

recontextual_chain = contextual_prompt | llm

context_injecter = recontextual_chain | ensemble_retriever | RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})

In [104]:
recontextual_chain

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [102]:
a = RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})
b = a.invoke(result_list)

In [106]:
history = []

recontextual_chain = contextual_prompt | llm

In [95]:
inputs = ({
    **b,
    'chat_history': history,
    'input': 'Explain transformers',
    'question': 'Explain transformers'
})

In [96]:
inputs

{'context': '\nChunk_1:\n\nTransformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily understood if the two coils are wound on opposite sides of a \nmagnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, \nbut the magnetic coupling is not as good as with the shell-type construction. \n \n \nFigure 6-6  Core-type transformer \n \n \n \nFigure 6 -7 is a schema

In [109]:
history

[HumanMessage(content='Explain transformersz', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Transformers are practical applications of magnetically coupled coils used to transfer energy from one coil to another. They essentially work as AC devices, transferring energy through a change in flux to induce voltage in a coil. The main reasons for using transformers are to provide electrical isolation between the source and load, as well as to change voltage and current levels.\n\nTransformers use a magnetic core to improve coupling between the coils. The cores can be made from materials like magnetically soft ferrites or iron alloys, depending on the frequency of the transformer. High-frequency transformers often use ferrites, while power frequencies use iron alloys.\n\nThere are different types of transformers, including shell-type and toroidal transformers. Shell-type transformers have a central limb that carries twice the flux as the outer limbs and are used for small

In [42]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [110]:
history_aware_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [116]:
get_session_history('17egw2737').messages

[]

In [ ]:
import uuid

def temp_pipeline():

    session_id = str(uuid.uuid4())[:8]
    print(f"Session ID: {session_id}")
    
    history = get_session_history(session_id)

    print(f"\nModel {MODEL_NAME} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")
    while True:

        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended. Have a good day.")
            break
        
        recontextual_chain = contextual_prompt | llm
        rephrased_question = recontextual_chain.invoke(
            {'chat_history': history.messages,
             'input': user_input})
        
        print(rephrased_question)

        context_injection = (ensemble_retriever | RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})).invoke(rephrased_question)

        print("Context:\n", context_injection['context'])  # Preview the first 500 chars
        print("Metadata:\n", context_injection['metadata'])
        
        response = history_aware_chain.invoke(
            {**context_injection,
            'input': user_input,
            'question': rephrased_question},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"LLM: {response}\n")

In [126]:
temp_pipeline()

Session ID: 3253ba96

Model llama3.2 has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.
Here's a reformulated version of the question:

What are transformers?
Context:
 
Chunk_1:

6.1.2 Practical aspects: 
The cores for high -frequency transformers are often made from magnetically soft ferrites, which are 
electrical insulators. For power frequencies, the cores are made from an iron alloy such as silicon steel. 
These materials have better magnetic properties than ferrites, but they are also electrical conductors. If 
the core were made of solid steel, currents – known as eddy currents – would be induced in the core by 
the alternating magnetic flux.  Steel tra
Metadata:
 This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 51, 52, 1

ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf, pages 15, 14, 13

ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf, pages 12
LLM: Transformers are 

In [43]:
recontextualiser_chain = contextual_prompt | llm
recontextualizer = RunnableWithMessageHistory(recontextualiser_chain, get_session_history)

In [44]:
type(recontextualizer)

langchain_core.runnables.history.RunnableWithMessageHistory

In [51]:
from langchain_core.output_parsers import StrOutputParser

rephrase_chain = contextual_prompt | llm | StrOutputParser()

In [ ]:
rephrase_chain.invoke(
    {'chat_history': [answer1],
     'input': "What materials are they made of?"}
)

'No rephrasing is needed; the question remains:\n\nWhat materials are transformers made of?'

In [58]:
type(answer1)

str

In [ ]:
import uuid

def pipeline(llm = llm):

    session_id = uuid.uuid4()[:8]
    print(f"\nModel {MODEL_NAME} with {session_id} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")

    while True:

        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended. Have a good day.")
            break

        reframed_question = recontextualizer.invoke(user_input, session_id)
        retrieved_docs = ensemble_retriever.invoke(reframed_question)
        context = content_parser(retrieved_docs)
        assoc_metadata = metadata_parser(retrieved_docs)

        chain.invoke(
            {"context": context,
            "metadata": assoc_metadata,
            "input": reframed_question,
            "question": reframed_question}
        )

In [ ]:
type(chain)

In [ ]:
pipeline()